In [1]:
import os

In [3]:
%pwd

'c:\\Users\\osama\\OneDrive\\Desktop\\Usama_dev\\Chicken_disease_classification\\research'

In [4]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\osama\\OneDrive\\Desktop\\Usama_dev\\Chicken_disease_classification'

In [7]:
# Entity is nothing but it is the return type of the function
from dataclasses import dataclass
from pathlib import Path

In [10]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source:str
    local_data_file:Path
    unzip_dir:Path


In [11]:
# To read the yaml file you first need to mention it in the constant directory (constructor)
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\osama\OneDrive\Desktop\Usama_dev\chiken_disease_env\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\osama\AppData\Local\Temp\ipykernel_13160\1582528049.py", line 3, in <module>
    from src.cnnClassifier.utils.common import read_yaml,create_directories
  File "c:\Users\osama\OneDrive\Desktop\Usama_dev\Chicken_disease_classification\src\cnnClassifier\utils\common.py", line 3, in <module>
    import yaml
ModuleNotFoundError: No module named 'yaml'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\osama\OneDrive\Desktop\Usama_dev\chiken_disease_env\Lib\site-packages\IPython\core\interactiveshell.py", line 2173, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\osama\OneDrive\Desktop\Us